In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Reads food form
food = pd.read_csv("food.csv")

# Reads food_nutrient id pairing form
food_nutri = pd.read_csv("food_nutrient.csv")
food_nutri = food_nutri.drop(columns = ['min', 'max', 'median', 'footnote', 'min_year_acquired'])

# Reads food nutrients conversion
food_nutri_conv = pd.read_csv("food_nutrient_conversion_factor.csv")

# A USELESS FORM
food_nutri_src = pd.read_csv("food_nutrient_source.csv") 

# Nutrient form
nutri = pd.read_csv("Food Data/nutrient.csv")


/Users/daniel/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
# Categories 
categ = pd.read_csv("food_category.csv")

In [3]:
display(food_nutri.shape, food.shape)
len(food_nutri['fdc_id'].unique())
food_nutri.head()

(4800957, 6)

(292060, 5)

,id,fdc_id,nutrient_id,amount,data_points,derivation_id
0,4178832,346049,1079,0.00,NaN,75.0
1,4178833,346049,1087,0.00,NaN,75.0
2,4178834,346049,1089,2.57,NaN,75.0
3,4178835,346049,1104,0.00,NaN,75.0
4,4178836,346049,1162,0.00,NaN,75.0


In [4]:
# Creates new column lower_name of lower letters
food['brand_name'] = food['description'].str.extract(pat = '([A-Z]{2,})*') 
food['lower_name'] = food['description'].str.extract(pat = '([^\d]+)') 
food['lower_name'] = food['lower_name'].str.strip().str.replace(r',$', '').str.lower()

In [5]:
# TODO creates foodname
food['description_comma'] = food['description'] + ','
food['food_name'] = food['description_comma'].str.extract(pat = '[A-Z]{2,}\w(.*), \d')


In [6]:
# Create food nutrient pairs with nutrient names 
food_nutri_pair = food_nutri[['fdc_id', 'nutrient_id', 'amount']]
food_nutri_pair = food_nutri_pair[food_nutri_pair['amount'] > 0.0]
food_nutri_pair['amount'] = np.round(food_nutri_pair['amount'], 2)
food_nutri_name = food_nutri_pair.merge(nutri[['id', 'name']], left_on = 'nutrient_id', right_on = 'id', how = 'left').drop(columns = ['id'])

In [7]:
# food_nutri_name['list_name_amount']
food_nutri_name['name_amount'] = food_nutri_name[['name', 'amount']].values.tolist()
food_nutri_name['name_amount'].head()

0                       [Iron, Fe, 2.57]
1                        [Protein, 25.0]
2             [Total lipid (fat), 14.29]
3    [Carbohydrate, by difference, 3.57]
4                   [Potassium, K, 27.0]
Name: name_amount, dtype: object

In [8]:
# Creates a dictionary of nutrients for each food item
food_nutri_dict = food_nutri_name.groupby('fdc_id')['name_amount'].agg(lambda s: dict(zip(s.map(lambda x: x[0]).tolist()
                                                                             , s.map(lambda y: y[1]).tolist())))

food_nutri_dict_df = pd.DataFrame({'nutrients': food_nutri_dict})
food_nutri_dict_df.head()

,nutrients
fdc_id,
167512,"{'Protein': 5.88, 'Ash': 3.5, 'Energy': 307.0,..."
167513,"{'Protein': 4.34, 'Ash': 3.08, 'Energy': 330.0..."
167514,"{'Energy': 377.0, 'Sodium, Na': 2182.0, 'Total..."
167515,"{'Ash': 1.6, 'Water': 42.6, 'Energy': 232.0, '..."
167516,"{'Fatty acids, total saturated': 1.9, 'Vitamin..."


In [9]:
display(food_nutri_dict.shape, food.shape)

(281837,)

(292060, 9)

In [10]:
# Add the nutrients back to the food table
# Run this for only ONE TIME
food = food.merge(food_nutri_dict_df, left_on = 'fdc_id', right_on = 'fdc_id', how = 'left')

In [11]:

food_json = food[['lower_name', 'nutrients']].set_index(food['fdc_id'])
# Export = food_json.to_json (r'/Users/daniel/Documents/College/FunnelFoods/food_nutrients.json', orient='index')

In [12]:
# list of unique nutrients: nutri df
food_nutrient_info = food_json.rename(columns = {'lower_name': 'name'})
# Export = food_nutrient_info.to_csv(r'/Users/daniel/Documents/College/FunnelFoods/food_nutrients_dict.csv')

In [141]:
food_nutri_id_pair = food_nutri[['fdc_id', 'nutrient_id']][food_nutri['amount'] > 0]
food_nutri_id_pair = food_nutri_id_pair.reset_index(drop = True)

# Export = food_nutri_id_pair.to_csv(r'/Users/daniel/Documents/College/FunnelFoods/food_nutrient_id_pair.csv')

In [ ]:
b_fdc_list = [320385,
 320397,
 334782,
 334797,
 334819,
 334829,
 334857,
 334874,
 334924,
 334941,
 334973,
 334991,
 335022,
 335039,
 335082,
 335094]
b_fdc_list

food.loc[food['fdc_id'].isin(b_fdc_list), 'lower_name'] = food[food['fdc_id'].isin(b_fdc_list)]['description'].str.lower().str.extract(r'b12, b6, b3, b2 - (.+)')[0]

In [142]:
temp = food[~np.isnan(food['food_category_id'])]
food_categorizied_nutrients = temp[~temp['nutrients'].isnull()]
food_categorizied_nutrients.head()

,fdc_id,data_type,description,food_category_id,publication_date,brand_name,lower_name,description_comma,food_name,nutrients
260001,321510,sub_sample_food,Beans snap canned drained,11.0,2019-04-01,NaN,beans snap canned drained,"Beans snap canned drained,",NaN,"{'Calcium, Ca': 35.0, 'Copper, Cu': 0.04, 'Iro..."
260002,321511,sub_sample_food,"Beans, snap, canned, drained - NFY1211Z9",11.0,2019-04-01,NaN,"beans, snap, canned, drained - nfy","Beans, snap, canned, drained - NFY1211Z9,",NaN,"{'Ash': 0.94, 'Total lipid (fat)': 0.43, 'Wate..."
260004,321519,sub_sample_food,Beans snap canned drained,11.0,2019-04-01,NaN,beans snap canned drained,"Beans snap canned drained,",NaN,"{'Calcium, Ca': 37.0, 'Copper, Cu': 0.04, 'Iro..."
260005,321520,sub_sample_food,"Beans, snap, canned, drained - NFY1211ZM",11.0,2019-04-01,NaN,"beans, snap, canned, drained - nfy","Beans, snap, canned, drained - NFY1211ZM,",NaN,"{'Ash': 1.05, 'Total lipid (fat)': 0.38, 'Wate..."
260007,321525,sub_sample_food,Beans snap canned drained,11.0,2019-04-01,NaN,beans snap canned drained,"Beans snap canned drained,",NaN,"{'Calcium, Ca': 34.0, 'Copper, Cu': 0.06, 'Iro..."


In [42]:
food_nutri_amount = food_nutri[['fdc_id', 'nutrient_id', 'amount']][food_nutri['amount'] > 0]
# Export = food_nutri_amount.to_csv(r'food_nutrient_amount.csv')

In [143]:
food_categorized_nurtrients_w_name = food_categorizied_nutrients.merge(categ, left_on = 'food_category_id', right_on = 'id', how= 'left')[
    ['fdc_id', 'lower_name', 'nutrients','food_category_id', 'description_y', ]
        ].rename(columns = {
    'lower_name':'food_name', 
    'description_y':'category_name',
})

In [144]:
food_categorized_nurtrients_w_name.head()

,fdc_id,food_name,nutrients,food_category_id,category_name
0,321510,beans snap canned drained,"{'Calcium, Ca': 35.0, 'Copper, Cu': 0.04, 'Iro...",11.0,Vegetables and Vegetable Products
1,321511,"beans, snap, canned, drained - nfy","{'Ash': 0.94, 'Total lipid (fat)': 0.43, 'Wate...",11.0,Vegetables and Vegetable Products
2,321519,beans snap canned drained,"{'Calcium, Ca': 37.0, 'Copper, Cu': 0.04, 'Iro...",11.0,Vegetables and Vegetable Products
3,321520,"beans, snap, canned, drained - nfy","{'Ash': 1.05, 'Total lipid (fat)': 0.38, 'Wate...",11.0,Vegetables and Vegetable Products
4,321525,beans snap canned drained,"{'Calcium, Ca': 34.0, 'Copper, Cu': 0.06, 'Iro...",11.0,Vegetables and Vegetable Products


In [146]:
food_categorized_nurtrients_w_name[food_categorized_nurtrients_w_name['fdc_id'].isin(b_fdc_list)]
# Export = food_categorized_nurtrients_w_name.to_csv(r'food_categorized_nurtrients_w_name.csv')